In [2]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [5]:
world_map = folium.Map()
world_map

In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [10]:
folium.map.FeatureGroup()

In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

circle1 = folium.Circle(location=[28.562302, -80.577356], radius=1000, color='#d35400', fill=True).add_child(folium.Popup('CCAFS LC-40'))
circle2 = folium.Circle(location=[28.563197, -80.576820], radius=1000, color='#d35400', fill=True).add_child(folium.Popup('CCAFS SLC-40'))

marker = folium.map.Marker(location=[28.562302, -80.577356], icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS LC-40',))
marker = folium.map.Marker(location=[28.563197, -80.576820], icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS SLC-40',))

site_map.add_child(circle)
site_map.add_child(marker)



In [12]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [13]:
marker_cluster = MarkerCluster()

In [14]:
 # Define a function to assign colors based on class value
def assign_marker_color(class_value):
    if class_value == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to create the new column
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Display the dataframe to check the new column
print(spacex_df)


     Launch Site        Lat        Long  class marker_color
0    CCAFS LC-40  28.562302  -80.577356      0          red
1    CCAFS LC-40  28.562302  -80.577356      0          red
2    CCAFS LC-40  28.562302  -80.577356      0          red
3    CCAFS LC-40  28.562302  -80.577356      0          red
4    CCAFS LC-40  28.562302  -80.577356      0          red
5    CCAFS LC-40  28.562302  -80.577356      0          red
6    CCAFS LC-40  28.562302  -80.577356      0          red
7    CCAFS LC-40  28.562302  -80.577356      0          red
8    CCAFS LC-40  28.562302  -80.577356      0          red
9    CCAFS LC-40  28.562302  -80.577356      0          red
10   CCAFS LC-40  28.562302  -80.577356      0          red
11   CCAFS LC-40  28.562302  -80.577356      0          red
12   CCAFS LC-40  28.562302  -80.577356      0          red
13   CCAFS LC-40  28.562302  -80.577356      0          red
14   CCAFS LC-40  28.562302  -80.577356      0          red
15   CCAFS LC-40  28.562302  -80.577356 

In [15]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [19]:
await piplite.install(['haversine'])

In [20]:
from haversine import haversine, Unit

# Example dataframe with launch site and proximities
data = {
    'site_name': ['Launch Site', 'Proximity 1', 'Proximity 2'],
    'latitude': [28.5623, 28.5634, 28.5645],
    'longitude': [-80.5774, -80.5785, -80.5796]
}
spacex_df = pd.DataFrame(data)

# Coordinates of the launch site
launch_site_coords = (spacex_df.loc[spacex_df['site_name'] == 'Launch Site', 'latitude'].values[0],
                      spacex_df.loc[spacex_df['site_name'] == 'Launch Site', 'longitude'].values[0])

# Function to calculate distance from launch site
def calculate_distance(row):
    proximity_coords = (row['latitude'], row['longitude'])
    return haversine(launch_site_coords, proximity_coords, unit=Unit.KILOMETERS)

# Apply the function to calculate distances
spacex_df['distance_to_launch_site'] = spacex_df.apply(calculate_distance, axis=1)

# Display the dataframe with distances
print(spacex_df)


     site_name  latitude  longitude  distance_to_launch_site
0  Launch Site   28.5623   -80.5774                 0.000000
1  Proximity 1   28.5634   -80.5785                 0.162793
2  Proximity 2   28.5645   -80.5796                 0.325586


In [21]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [22]:
import haversine as hs

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the distance between two lat/lon pairs using the Haversine formula."""
    loc1 = (lat1, lon1)
    loc2 = (lat2, lon2)
    return hs.haversine(loc1, loc2)  # returns distance in kilometers

def find_closest_coastline(lat, lon, coastline_points):
    """Find the closest coastline point to the given lat/lon coordinates."""
    min_distance = float('inf')
    closest_point = None
    
    for coast_lat, coast_lon in coastline_points:
        distance = haversine_distance(lat, lon, coast_lat, coast_lon)
        if distance < min_distance:
            min_distance = distance
            closest_point = (coast_lat, coast_lon)
    
    return closest_point, min_distance

In [23]:
import folium
from folium.features import DivIcon
import haversine as hs

# Function to calculate haversine distance between two points
def haversine_distance(lat1, lon1, lat2, lon2):
    loc1 = (lat1, lon1)
    loc2 = (lat2, lon2)
    return hs.haversine(loc1, loc2)

# Function to find closest coastline point
def find_closest_coastline(lat, lon, coastline_points):
    min_distance = float('inf')
    closest_point = None
    
    for coast_lat, coast_lon in coastline_points:
        distance = haversine_distance(lat, lon, coast_lat, coast_lon)
        if distance < min_distance:
            min_distance = distance
            closest_point = (coast_lat, coast_lon)
    
    return closest_point, min_distance

# Coordinates for the launch site
launch_site_lat = 28.56367
launch_site_lon = -80.57163

# Example coastline points (replace this with real coastline data)
coastline_points = [
    (28.1, -80.6),
    (28.3, -80.5),
    (28.7, -80.3),  # Example points
]

# Find the closest coastline point
closest_coastline, distance_to_coastline = find_closest_coastline(launch_site_lat, launch_site_lon, coastline_points)

# Create a folium map centered around the launch site
launch_site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add a marker for the launch site
folium.Marker([launch_site_lat, launch_site_lon], popup="Launch Site").add_to(launch_site_map)

# Add a marker for the closest coastline point
folium.Marker(
    closest_coastline,
    popup="Closest Coastline",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(launch_site_map)

# Add a distance marker to show the distance between the launch site and the closest coastline
distance_marker = folium.Marker(
    closest_coastline,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_coastline),
    )
)
distance_marker.add_to(launch_site_map)

# Display the map
launch_site_map.save("launch_site_map.html")
launch_site_map

In [24]:
site_map.add_child(lines)

<class 'NameError'>: name 'lines' is not defined